#와 오토인코더...

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,id,site,timestamp,vibration,sound,WaterPressure
0,1,2.0,2023-08-01T14:43:33.000Z,0,38,188
1,2,2.0,2023-08-01T14:43:35.000Z,0,38,150
2,3,2.0,2023-08-01T14:43:36.000Z,0,38,38
3,4,2.0,2023-08-01T14:43:37.000Z,0,38,167
4,5,2.0,2023-08-01T14:43:38.000Z,0,37,124


In [ ]:
df.dtypes

id                 int64
site             float64
timestamp         object
vibration          int64
sound              int64
WaterPressure      int64
dtype: object

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

df = pd.DataFrame(df, columns=['id', 'site', 'timestamp', 'vibration', 'sound', 'WaterPressure'])


df['timestamp'] = pd.to_datetime(df['timestamp'])


df['timestamp'] = df['timestamp'].values.astype(np.int64) // 10 ** 9


print(df)

          id  site   timestamp  vibration  sound  WaterPressure
0          1   2.0  1690901013          0     38            188
1          2   2.0  1690901015          0     38            150
2          3   2.0  1690901016          0     38             38
3          4   2.0  1690901017          0     38            167
4          5   2.0  1690901018          0     37            124
...      ...   ...         ...        ...    ...            ...
41693  41694   1.0  1690994239          0    475             96
41694  41695   1.0  1690994239          0    490            487
41695  41696   1.0  1690994240          0    480            481
41696  41697   1.0  1690994241          0    483            180
41697  41698   1.0  1690994242          0    477             96

[41698 rows x 6 columns]


#도전..!

##1차 Relu - Sigmoid

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 데이터 전처리
# 정상(normal) 누수(leak)
normal_data = df[:26410]
leak_data = df[26410:]

# 정상(normal) 데이터는 0, 누수(leak) 데이터는 1
normal_data_with_label = np.hstack((normal_data, np.zeros((normal_data.shape[0], 1), dtype=int)))
leak_data_with_label = np.hstack((leak_data, np.ones((leak_data.shape[0], 1), dtype=int)))

# 정상과 누수 데이터를 합치기
data_with_label = np.vstack((normal_data_with_label, leak_data_with_label))

# 데이터와 라벨 분리
X = data_with_label[:, 2:]  # timestamp, vibration, sound, WaterPressure
y = data_with_label[:, -1]  # 라벨은 마지막 열

# Min Max......
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# train,test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 파라미터
input_dim = X_train.shape[1]  
encoding_dim = 32  

# 오토인코더 모델 구축
input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_layer)
decoder = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)

# 오토인코더 모델 컴파일
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# checkpoint_cb
checkpoint_filepath = 'autoencoder_checkpoint.h5'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# early_stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   
    patience=2,           
    restore_best_weights=True  
)

# 오토인코더 학습
autoencoder.fit(
    X_train, X_train,
    epochs=1000,        
    batch_size=32,
    validation_data=(X_test, X_test),
    callbacks=[checkpoint_cb, early_stopping_cb]
)


reconstructed_X_test = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructed_X_test, 2), axis=1)
threshold = np.percentile(mse, 95)  
predicted_labels = (mse > threshold).astype(int)  


Epoch 1/1000
1041/1043 [============================>.] - ETA: 0s - loss: 0.0339
Epoch 1: val_loss improved from inf to 0.00165, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 4s 3ms/step - loss: 0.0339 - val_loss: 0.0016
Epoch 2/1000
1040/1043 [============================>.] - ETA: 0s - loss: 8.9641e-04
Epoch 2: val_loss improved from 0.00165 to 0.00056, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 2s 2ms/step - loss: 8.9581e-04 - val_loss: 5.5636e-04
Epoch 3/1000
1017/1043 [============================>.] - ETA: 0s - loss: 4.4373e-04
Epoch 3: val_loss improved from 0.00056 to 0.00034, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 2s 2ms/step - loss: 4.4157e-04 - val_loss: 3.4142e-04
Epoch 4/1000
1028/1043 [============================>.] - ETA: 0s - loss: 2.7302e-04
Epoch 4: val_loss improved from 0.00034 to 0.00021, saving model to autoencoder_checkpoint.h5
104

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 평가 예시
accuracy = accuracy_score(y_test, predicted_labels)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.6876498800959233
Precision: 1.0
Recall: 0.13798808735936466
F1 Score: 0.24251235824367548


##2차 Relu - Relu


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 데이터 전처리
# 정상(normal) 누수(leak)
normal_data = df[:26410]
leak_data = df[26410:]

# 정상(normal) 데이터는 0, 누수(leak) 데이터는 1
normal_data_with_label = np.hstack((normal_data, np.zeros((normal_data.shape[0], 1), dtype=int)))
leak_data_with_label = np.hstack((leak_data, np.ones((leak_data.shape[0], 1), dtype=int)))

# 정상과 누수 데이터를 합치기
data_with_label = np.vstack((normal_data_with_label, leak_data_with_label))

# 데이터와 라벨 분리
X = data_with_label[:, 2:]  # timestamp, vibration, sound, WaterPressure
y = data_with_label[:, -1]  # 라벨은 마지막 열

# Min Max......
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# train,test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 파라미터
input_dim = X_train.shape[1]  
encoding_dim = 32  

# 오토인코더 모델 구축
input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_layer)
decoder = tf.keras.layers.Dense(input_dim, activation='relu')(encoder)
autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)

# 오토인코더 모델 컴파일
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# checkpoint_cb
checkpoint_filepath = 'autoencoder_checkpoint.h5'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# early_stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   
    patience=2,           
    restore_best_weights=True  
)

# 오토인코더 학습
autoencoder.fit(
    X_train, X_train,
    epochs=1000,        
    batch_size=32,
    validation_data=(X_test, X_test),
    callbacks=[checkpoint_cb, early_stopping_cb]
)


reconstructed_X_test = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructed_X_test, 2), axis=1)
threshold = np.percentile(mse, 95)  
predicted_labels = (mse > threshold).astype(int)  


Epoch 1/1000
1031/1043 [============================>.] - ETA: 0s - loss: 0.0959
Epoch 1: val_loss improved from inf to 0.08176, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 3s 2ms/step - loss: 0.0957 - val_loss: 0.0818
Epoch 2/1000
1025/1043 [============================>.] - ETA: 0s - loss: 0.0823
Epoch 2: val_loss improved from 0.08176 to 0.08169, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 2s 2ms/step - loss: 0.0823 - val_loss: 0.0817
Epoch 3/1000
1021/1043 [============================>.] - ETA: 0s - loss: 0.0822
Epoch 3: val_loss improved from 0.08169 to 0.08168, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 2s 2ms/step - loss: 0.0823 - val_loss: 0.0817
Epoch 4/1000
1024/1043 [============================>.] - ETA: 0s - loss: 0.0822
Epoch 4: val_loss improved from 0.08168 to 0.08167, saving model to autoencoder_checkpoint.h5
1043/1043 [====================

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 평가 예시
accuracy = accuracy_score(y_test, predicted_labels)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.6595923261390887
Precision: 0.7194244604316546
Recall: 0.09927200529450694
F1 Score: 0.17446932247746436


##3차 Softmax - Sigmoid

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 데이터 전처리
# 정상(normal) 누수(leak)
normal_data = df[:26410]
leak_data = df[26410:]

# 정상(normal) 데이터는 0, 누수(leak) 데이터는 1
normal_data_with_label = np.hstack((normal_data, np.zeros((normal_data.shape[0], 1), dtype=int)))
leak_data_with_label = np.hstack((leak_data, np.ones((leak_data.shape[0], 1), dtype=int)))

# 정상과 누수 데이터를 합치기
data_with_label = np.vstack((normal_data_with_label, leak_data_with_label))

# 데이터와 라벨 분리
X = data_with_label[:, 2:]  # timestamp, vibration, sound, WaterPressure
y = data_with_label[:, -1]  # 라벨은 마지막 열

# Min Max......
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# train,test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 파라미터
input_dim = X_train.shape[1]  
encoding_dim = 32  

# 오토인코더 모델 구축
input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(encoding_dim, activation='softmax')(input_layer)
decoder = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)

# 오토인코더 모델 컴파일
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# checkpoint_cb
checkpoint_filepath = 'autoencoder_checkpoint.h5'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# early_stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   
    patience=2,           
    restore_best_weights=True  
)

# 오토인코더 학습
autoencoder.fit(
    X_train, X_train,
    epochs=1000,        
    batch_size=32,
    validation_data=(X_test, X_test),
    callbacks=[checkpoint_cb, early_stopping_cb]
)


reconstructed_X_test = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructed_X_test, 2), axis=1)
threshold = np.percentile(mse, 95)  
predicted_labels = (mse > threshold).astype(int)  


Epoch 1/1000
1021/1043 [============================>.] - ETA: 0s - loss: 0.1233
Epoch 1: val_loss improved from inf to 0.05535, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 3s 2ms/step - loss: 0.1219 - val_loss: 0.0554
Epoch 2/1000
1028/1043 [============================>.] - ETA: 0s - loss: 0.0302
Epoch 2: val_loss improved from 0.05535 to 0.01613, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 3s 3ms/step - loss: 0.0300 - val_loss: 0.0161
Epoch 3/1000
1038/1043 [============================>.] - ETA: 0s - loss: 0.0121
Epoch 3: val_loss improved from 0.01613 to 0.00934, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 2s 2ms/step - loss: 0.0121 - val_loss: 0.0093
Epoch 4/1000
1032/1043 [============================>.] - ETA: 0s - loss: 0.0079
Epoch 4: val_loss improved from 0.00934 to 0.00657, saving model to autoencoder_checkpoint.h5
1043/1043 [====================

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 평가 예시
accuracy = accuracy_score(y_test, predicted_labels)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.6382494004796163
Precision: 0.5059952038369304
Recall: 0.06982131039046989
F1 Score: 0.12271009014248327


##4차 Leaky ReLU - Sigmoid

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 데이터 전처리
# 정상(normal) 누수(leak)
normal_data = df[:26410]
leak_data = df[26410:]

# 정상(normal) 데이터는 0, 누수(leak) 데이터는 1
normal_data_with_label = np.hstack((normal_data, np.zeros((normal_data.shape[0], 1), dtype=int)))
leak_data_with_label = np.hstack((leak_data, np.ones((leak_data.shape[0], 1), dtype=int)))

# 정상과 누수 데이터를 합치기
data_with_label = np.vstack((normal_data_with_label, leak_data_with_label))

# 데이터와 라벨 분리
X = data_with_label[:, 2:]  # timestamp, vibration, sound, WaterPressure
y = data_with_label[:, -1]  # 라벨은 마지막 열

# Min Max......
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# train,test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 파라미터
input_dim = X_train.shape[1]  
encoding_dim = 32  

# 오토인코더 모델 구축
input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(encoding_dim, activation='LeakyReLU')(input_layer)
decoder = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)

# 오토인코더 모델 컴파일
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# checkpoint_cb
checkpoint_filepath = 'autoencoder_checkpoint.h5'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# early_stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   
    patience=2,           
    restore_best_weights=True  
)

# 오토인코더 학습
autoencoder.fit(
    X_train, X_train,
    epochs=1000,        
    batch_size=32,
    validation_data=(X_test, X_test),
    callbacks=[checkpoint_cb, early_stopping_cb]
)


reconstructed_X_test = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructed_X_test, 2), axis=1)
threshold = np.percentile(mse, 95)  
predicted_labels = (mse > threshold).astype(int)  


Epoch 1/1000
1034/1043 [============================>.] - ETA: 0s - loss: 0.0326
Epoch 1: val_loss improved from inf to 0.00118, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 4s 3ms/step - loss: 0.0323 - val_loss: 0.0012
Epoch 2/1000
1017/1043 [============================>.] - ETA: 0s - loss: 6.8692e-04
Epoch 2: val_loss improved from 0.00118 to 0.00044, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 3s 3ms/step - loss: 6.8052e-04 - val_loss: 4.4308e-04
Epoch 3/1000
1019/1043 [============================>.] - ETA: 0s - loss: 3.4244e-04
Epoch 3: val_loss improved from 0.00044 to 0.00025, saving model to autoencoder_checkpoint.h5
1043/1043 [==============================] - 2s 2ms/step - loss: 3.4035e-04 - val_loss: 2.5324e-04
Epoch 4/1000
1032/1043 [============================>.] - ETA: 0s - loss: 2.0073e-04
Epoch 4: val_loss improved from 0.00025 to 0.00016, saving model to autoencoder_checkpoint.h5
104

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 평가 예시
accuracy = accuracy_score(y_test, predicted_labels)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.6619904076738609
Precision: 0.7434052757793765
Recall: 0.10258107213765719
F1 Score: 0.18028496656004653


In [ ]:
Accuracy: 0.668705035971223
Precision: 0.8105515587529976
Recall: 0.11184645929847783
F1 Score: 0.19656876999127654